# Understanding Columnar Storage Formats: A Practical Guide

This notebook provides a comprehensive exploration of columnar storage formats, focusing on their advantages and practical applications in data engineering. Through hands-on examples using Polars and Apache Arrow, we'll demonstrate why columnar formats like Parquet are the industry standard for analytical workloads.

**Author:** Data Engineering Team  
**Last Modified:** September 15, 2025

## Prerequisites
- Basic understanding of row-oriented formats (CSV, JSON)
- Familiarity with DataFrame libraries (Pandas or Polars)
- Basic Python programming knowledge

## Table of Contents
1. [Setup and Data Generation](#setup)
   - Library imports
   - Sample dataset creation
2. [CSV vs Parquet Format Comparison](#comparison)
   - File size analysis
   - Read performance
   - Memory usage patterns
3. [Column Pruning Performance](#pruning)
   - Column selection efficiency
   - I/O optimization demonstration
4. [Compression Analysis](#compression)
   - Compression ratio comparison
   - Data type grouping benefits
5. [Schema Evolution](#schema)
   - Adding/removing columns
   - Data type modifications
6. [Real-world Query Benchmarks](#benchmarks)
   - NYC Taxi dataset analysis
   - Practical performance testing

## Setup and Configuration

First, let's import the necessary libraries and set up our configuration. We'll be using:
- **Polars**: For high-performance data manipulation
- **pyarrow**: For Apache Arrow functionality and Parquet support
- **time**: For performance measurements
- **os**: For file operations
- **numpy**: For numerical operations and random data generation

### Install dependencies

In [ ]:
# !uv pip install ".[base, polars]"

### Setup

In [2]:
# Import required libraries
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from pathlib import Path
import time
from datetime import datetime, timedelta

# Configuration
SEED = 42
np.random.seed(SEED)

# File paths
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
CSV_PATH = DATA_DIR / "sample_data.csv"
PARQUET_PATH = DATA_DIR / "sample_data.parquet"

# Dataset parameters
N_ROWS = 100_000
N_COLS = 50

## Understanding Row vs Column-Oriented Storage

Before diving into the implementation, let's understand the fundamental difference between row and column-oriented storage using a simple analogy:

### The Phone Book Analogy 📱

Imagine you have two different versions of a phone book:

1. **Traditional Phone Book (Row-oriented)**
   - Each entry contains: (Name, Address, Phone Number)
   - Organized by complete records
   - Great for looking up all information about one person
   - Not efficient for finding "all phone numbers" or "all addresses"

2. **Specialized Index (Column-oriented)**
   - Separate lists for Names, Addresses, and Phone Numbers
   - Each type of data stored together
   - Perfect for questions like "list all phone numbers"
   - Better compression (similar data stored together)

This is exactly how row-oriented formats (like CSV) and columnar formats (like Parquet) differ in storing data. Let's see this in practice!

## Data Generation

Let's create a sample dataset with various data types to demonstrate the benefits of columnar storage. Our dataset will have:
- Numeric columns (integers and floats)
- Categorical columns
- DateTime columns
- Text columns

This variety of data types will help us showcase how columnar storage handles different types of data efficiently.

In [3]:
def generate_sample_data(n_rows: int, n_cols: int) -> pl.DataFrame:
    """
    Generate a sample DataFrame with various data types.
    
    Args:
        n_rows: Number of rows to generate
        n_cols: Total number of columns to generate (distributed across types)
        
    Returns:
        pl.DataFrame: Generated sample data
    """
    # Calculate number of columns per type
    n_per_type = n_cols // 4  # We'll have 4 types of columns
    
    # Generate numeric columns (integers)
    int_cols = {
        f"int_col_{i}": np.random.randint(0, 1000000, n_rows)
        for i in range(n_per_type)
    }
    
    # Generate float columns
    float_cols = {
        f"float_col_{i}": np.random.normal(0, 1, n_rows)
        for i in range(n_per_type)
    }
    
    # Generate categorical columns
    categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    cat_cols = {
        f"cat_col_{i}": np.random.choice(categories, n_rows)
        for i in range(n_per_type)
    }
    
    # Generate datetime columns
    base_date = datetime(2023, 1, 1)
    date_cols = {
        f"date_col_{i}": [
            base_date + timedelta(days=np.random.randint(0, 365))
            for _ in range(n_rows)
        ]
        for i in range(n_per_type)
    }
    
    # Combine all columns
    data = {**int_cols, **float_cols, **cat_cols, **date_cols}
    
    # Create Polars DataFrame
    df = pl.DataFrame(data)
    return df

# Generate the sample dataset
print("Generating sample dataset...")
df = generate_sample_data(N_ROWS, N_COLS)
print(f"Generated dataset shape: {df.shape}")
print("\nDataset preview:")
df.head()

Generating sample dataset...
Generated dataset shape: (100000, 48)

Dataset preview:


int_col_0,int_col_1,int_col_2,int_col_3,int_col_4,int_col_5,int_col_6,int_col_7,int_col_8,int_col_9,int_col_10,int_col_11,float_col_0,float_col_1,float_col_2,float_col_3,float_col_4,float_col_5,float_col_6,float_col_7,float_col_8,float_col_9,float_col_10,float_col_11,cat_col_0,cat_col_1,cat_col_2,cat_col_3,cat_col_4,cat_col_5,cat_col_6,cat_col_7,cat_col_8,cat_col_9,cat_col_10,cat_col_11,date_col_0,date_col_1,date_col_2,date_col_3,date_col_4,date_col_5,date_col_6,date_col_7,date_col_8,date_col_9,date_col_10,date_col_11
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
121958,245279,201413,581575,256279,607614,228105,192391,508457,814701,410705,578370,0.525842,-0.468028,-0.298357,1.833869,-0.835418,-1.903113,-1.206475,-0.820048,0.745167,1.60712,-0.08369,0.611347,"""A""","""I""","""B""","""A""","""J""","""I""","""D""","""B""","""I""","""E""","""I""","""J""",2023-06-28 00:00:00,2023-11-16 00:00:00,2023-08-16 00:00:00,2023-12-28 00:00:00,2023-04-04 00:00:00,2023-11-29 00:00:00,2023-10-19 00:00:00,2023-11-23 00:00:00,2023-11-30 00:00:00,2023-06-26 00:00:00,2023-05-06 00:00:00,2023-11-30 00:00:00
671155,178356,359215,423821,975578,59029,511718,890571,771742,936108,202453,368175,-0.296441,1.518756,0.197392,-0.1359,-0.464159,1.477194,0.448394,0.046258,1.334597,-0.376881,1.158888,-1.449445,"""A""","""C""","""H""","""H""","""E""","""A""","""H""","""F""","""B""","""D""","""B""","""E""",2023-05-15 00:00:00,2023-04-12 00:00:00,2023-09-09 00:00:00,2023-12-23 00:00:00,2023-05-26 00:00:00,2023-02-27 00:00:00,2023-03-22 00:00:00,2023-12-26 00:00:00,2023-11-12 00:00:00,2023-03-24 00:00:00,2023-06-13 00:00:00,2023-03-08 00:00:00
131932,752233,19752,871751,713522,32471,872191,938080,43188,997191,244294,307420,-0.364775,-2.893514,0.898982,-0.515307,1.037464,2.622034,-0.640263,-0.072445,0.598887,-0.565942,0.133276,0.813489,"""C""","""E""","""A""","""F""","""E""","""A""","""C""","""A""","""A""","""H""","""H""","""E""",2023-05-11 00:00:00,2023-05-15 00:00:00,2023-10-26 00:00:00,2023-03-05 00:00:00,2023-09-26 00:00:00,2023-05-21 00:00:00,2023-10-12 00:00:00,2023-04-14 00:00:00,2023-04-13 00:00:00,2023-09-09 00:00:00,2023-10-16 00:00:00,2023-06-21 00:00:00
365838,895983,131058,438563,118669,953547,365077,338394,33506,261780,925517,410270,-1.616431,0.071071,0.606215,0.214326,-0.962773,-1.317677,-0.799817,0.591787,-0.172468,0.341324,0.303579,0.749992,"""A""","""I""","""C""","""I""","""B""","""I""","""B""","""B""","""G""","""H""","""F""","""I""",2023-07-01 00:00:00,2023-09-15 00:00:00,2023-05-28 00:00:00,2023-11-15 00:00:00,2023-05-02 00:00:00,2023-04-25 00:00:00,2023-06-16 00:00:00,2023-06-15 00:00:00,2023-10-11 00:00:00,2023-06-19 00:00:00,2023-03-02 00:00:00,2023-05-09 00:00:00
259178,63724,663550,720980,952984,307704,331649,687036,503917,998094,712215,624442,1.046499,1.004912,0.574196,-0.391352,1.990713,-1.239422,-0.103856,0.391309,-0.384701,0.168152,-1.219548,-0.904327,"""D""","""F""","""H""","""A""","""E""","""H""","""H""","""B""","""B""","""C""","""D""","""I""",2023-09-29 00:00:00,2023-10-10 00:00:00,2023-10-23 00:00:00,2023-04-25 00:00:00,2023-10-16 00:00:00,2023-04-06 00:00:00,2023-12-31 00:00:00,2023-11-12 00:00:00,2023-08-06 00:00:00,2023-09-23 00:00:00,2023-02-16 00:00:00,2023-03-23 00:00:00


## CSV vs Parquet Format Comparison

Now that we have our sample dataset, let's compare how it behaves when stored in CSV (row-oriented) versus Parquet (columnar) format. We'll examine:

1. File sizes on disk
2. Read performance
3. Memory usage

We'll write our dataset to both formats and then analyze the differences.

In [ ]:
from notebooks.util.profile import format_size

# Write to CSV and Parquet
print("Writing files...")
df.write_csv(CSV_PATH)
df.write_parquet(PARQUET_PATH)

# Compare file sizes
csv_size = Path(CSV_PATH).stat().st_size
parquet_size = Path(PARQUET_PATH).stat().st_size

print("\nFile size comparison:")
print(f"CSV size: {format_size(csv_size)}")
print(f"Parquet size: {format_size(parquet_size)}")
print(f"Compression ratio: {csv_size / parquet_size:.2f}x")

### Read Performance Comparison

Let's compare how fast we can read data from both formats. We'll measure:
1. Time to read the entire dataset
2. Memory usage when reading
3. Time to read specific columns (column pruning)

In [4]:
from notebooks.util.profile import measure_read_time

# Read complete files
print("Reading complete CSV file:")
@measure_read_time
def read_csv():
    return pl.read_csv(CSV_PATH)

csv_df = read_csv()

print("\nReading complete Parquet file:")
@measure_read_time
def read_parquet():
    return pl.read_parquet(PARQUET_PATH)

parquet_df = read_parquet()

Reading complete CSV file:
Time taken: 0.587 seconds
Memory usage: 169.50 MB

Reading complete Parquet file:
Time taken: 0.255 seconds
Memory usage: 54.21 MB


## Column Pruning Performance

One of the key advantages of columnar storage is its ability to read only the columns needed for a specific query. This is called "column pruning" and it can significantly improve performance for queries that only need a subset of columns.

Let's demonstrate this by reading different numbers of columns from both formats:

In [5]:
# Test column pruning with different numbers of columns
column_counts = [1, 5, 10, 25, N_COLS]  # Test with different numbers of columns

print("Testing column pruning performance:")
print("-" * 50)

for n_cols in column_counts:
    # Get a subset of columns
    columns = df.columns[:n_cols]
    
    print(f"\nReading {n_cols} columns:")
    
    print("\nFrom CSV:")
    @measure_read_time
    def read_csv_columns():
        return pl.read_csv(CSV_PATH, columns=columns)
    
    csv_subset = read_csv_columns()
    
    print("\nFrom Parquet:")
    @measure_read_time
    def read_parquet_columns():
        return pl.read_parquet(PARQUET_PATH, columns=columns)
    
    parquet_subset = read_parquet_columns()

Testing column pruning performance:
--------------------------------------------------

Reading 1 columns:

From CSV:
Time taken: 0.097 seconds
Memory usage: 62.60 MB

From Parquet:
Time taken: 0.008 seconds
Memory usage: 1.89 MB

Reading 5 columns:

From CSV:
Time taken: 0.094 seconds
Memory usage: 66.16 MB

From Parquet:
Time taken: 0.021 seconds
Memory usage: 4.93 MB

Reading 10 columns:

From CSV:
Time taken: 0.074 seconds
Memory usage: 71.88 MB

From Parquet:
Time taken: 0.057 seconds
Memory usage: 8.36 MB

Reading 25 columns:

From CSV:
Time taken: 0.112 seconds
Memory usage: 80.54 MB

From Parquet:
Time taken: 0.098 seconds
Memory usage: 20.93 MB

Reading 50 columns:

From CSV:
Time taken: 0.186 seconds
Memory usage: 158.00 MB

From Parquet:
Time taken: 0.136 seconds
Memory usage: 48.00 MB


## Schema Evolution and Compatibility

One of the key advantages of columnar formats like Parquet is their ability to handle schema changes gracefully. This is particularly important in real-world scenarios where data structures evolve over time. Let's demonstrate this with a practical example:

### Scenario: Evolving E-commerce Order Data

Imagine you're working with e-commerce order data that evolves over time:
1. Initially, you track basic order information
2. Later, you add customer satisfaction scores
3. Finally, you need to handle orders with multiple shipping addresses

This example will show how Parquet handles these changes seamlessly, while highlighting the challenges with CSV format.

In [ ]:
# Create initial order data
initial_orders = pl.DataFrame({
    'order_id': range(1, 6),
    'product': ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard'],
    'price': [1200, 800, 300, 400, 80],
    'date': ['2025-01-01', '2025-01-02', '2025-01-02', '2025-01-03', '2025-01-03']
})

# Save in both formats
initial_csv = DATA_DIR / "orders_v1.csv"
initial_parquet = DATA_DIR / "orders_v1.parquet"

initial_orders.write_csv(initial_csv)
initial_orders.write_parquet(initial_parquet)

print("Initial schema:")
print(initial_orders.schema)

# Phase 2: Add customer satisfaction scores (some missing)
updated_orders = initial_orders.with_columns([
    pl.Series('satisfaction_score', [5, 4, None, 5, None], dtype=pl.Float32)
])

# Save updated data
updated_csv = DATA_DIR / "orders_v2.csv"
updated_parquet = DATA_DIR / "orders_v2.parquet"

updated_orders.write_csv(updated_csv)
updated_orders.write_parquet(updated_parquet)

print("\nUpdated schema with satisfaction scores:")
print(updated_orders.schema)

# Phase 3: Add multiple shipping addresses (nested data)
final_orders = updated_orders.with_columns([
    pl.Series('shipping_addresses', [
        ['Home: 123 Main St'],
        ['Work: 456 Corp Ave', 'Home: 789 Side St'],
        ['Home: 321 Oak Rd'],
        ['Work: 654 Biz Blvd', 'Pickup: Store #12'],
        ['Home: 987 Pine St']
    ])
])

# Try to save in CSV (this might be problematic)
try:
    final_csv = DATA_DIR / "orders_v3.csv"
    final_orders.write_csv(final_csv)
    print("\nCSV writing succeeded (but lists are converted to strings)")
except Exception as e:
    print("\nCSV writing failed:", str(e))

# Save in Parquet (handles nested data naturally)
final_parquet = DATA_DIR / "orders_v3.parquet"
final_orders.write_parquet(final_parquet)

print("\nFinal schema with nested shipping addresses:")
print(final_orders.schema)

# Demonstrate backwards compatibility
print("\nReading old data with new schema (Parquet):")
old_with_new_schema = pl.read_parquet(
    initial_parquet, columns=final_orders.columns
)
print("\nMissing columns are handled gracefully:")
print(old_with_new_schema.columns)

print("\nReading new data with old schema (column pruning):")
new_with_old_cols = pl.read_parquet(
    final_parquet,
    columns=['order_id', 'product', 'price', 'date']
)
print("Only requested columns are loaded:")
print(new_with_old_cols.head())

## Key Advantages of Columnar Storage for Schema Evolution

The example above demonstrates several key advantages of columnar formats like Parquet over row-based formats like CSV:

1. **Handling Missing Data**
   - When we added satisfaction scores, Parquet efficiently handles NULL values with its built-in null support
   - CSV requires special handling and often uses placeholders like empty strings or "NA"

2. **Nested Data Structures**
   - Parquet natively supports complex data types (lists, maps, structs)
   - CSV has no native support for nested data - requires serialization (e.g., converting to strings)
   - This becomes crucial when your data naturally contains arrays or nested structures

3. **Schema Metadata**
   - Parquet files contain explicit schema information
   - CSV files have no schema - every reader must guess types
   - This means Parquet can ensure data consistency across files

4. **Backward/Forward Compatibility**
   - Old Parquet files remain readable when schema changes
   - New columns can be added without breaking existing code
   - Readers can request only the columns they understand

5. **Performance Impact**
   - When schema evolves, Parquet's column pruning still works
   - You only read the columns you need, regardless of schema changes
   - This maintains query performance even as schemas grow

These features make Parquet (and columnar formats in general) much better suited for real-world data applications where requirements and data structures evolve over time.